In [34]:
pip install jupyter-dash -q  #Installing the Jupyter Dash library which we would using for making our Dashboard.

In [35]:
#This section of the code is used to upload our .csv dataset to Google Colabo
from google.colab import files
uploaded = files.upload()

Saving tata.csv to tata (1).csv


In [36]:
#Importing the required modules form the respective Libraries (lines 3-9)

from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import pandas as pd
import numpy as np
from dash.dependencies import Output, Input
import plotly.graph_objects as go

#Through this section of the code we are reading the tata motors dataset and extracting the required information from it (lines 13-19)

data = pd.read_csv("tata.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
data.sort_values("Date", inplace=True)

options=list(data.columns)[1:5]
options.append('Open v/s Close')
options.append('High v/s Low')

#Here we are creating an instance of the Jupyter Dash Class
app = JupyterDash(__name__)

#The layout detrmines the looks of the application
app.layout = html.Div( #It represents a Division
    children=[
        html.Div( 
            children=[
                html.H1( #It represents a heading
                    children="Share Analysis", #Heading Contents
                    style={"color": "#FFFFFF","fontSize": "48px","fontWeight": "bold","marginTop": "32px",}, #These are the various styling options used
                ),
                html.P( #It represents a paragraph
                    children="This Dashboard analyses the price as well as volume traded of the shares of Tata Motors between 2016 and 2021.", #Paragraph Contents
                    style={"color": "#FFFFFF","margin": "4px auto","maxWidth": "384px",}, #These are the various styling options used
                ),
            ],
            style={"backgroundColor": "#222222","height": "288px","padding": "16px 0 0 0","textAlign": "center", #These are the various styling options used for the whole division 
},
        ),
        html.Div( #It represents a Division
            children=[
                html.Div(
                    children=[
                        html.Div(children="Trend of the Share", 
                                 style={"marginBottom": "6px","fontWeight": "bold","color": "#222222",}, #These are the various styling options used
                              ),
                        dcc.Dropdown( #It represents a dropdown menu
                            id="trend-filter", #It is the identifier
                            options=[{"label": trend, "value": trend} for trend in np.sort(options)], #These are the options of the dropdown menu
                            value="Open", #Default Value
                            clearable=False, # Means the field cannot be empty
                            style={"width": "256px","height": "47px","fontWeight": "bold",}, #These are the various styling options used
                        ),
                    ]
                ),

                html.Div( #It represents a Division
                    children=[
                        html.Div(
                            children="Duration",
                            style={"marginBottom": "6px","fontWeight": "bold","color": "#222222",}, #These are the various styling options used
                            ),
                        dcc.DatePickerRange( #It represents a Date Pick Menu
                            id="date-range", #It is the identifier
                            min_date_allowed=data.Date.min().date(), #Minimum Date of the Menu
                            max_date_allowed=data.Date.max().date(), #Maximum Date of the Menu
                            start_date=data.Date.min().date(), #Start Date of the Menu
                            end_date=data.Date.max().date(), #End Date of the Menu
                        ),
                    ]
                ),
            ],
            style={"height": "112px","width": "912px","display": "flex","justifyContent": "space-evenly","paddingTop": "24px","margin": "-80px auto 0 auto","backgroundColor": "#FFFFFF","boxShadow": "0 4px 6px 0 rgba(0, 0, 0, 0.18)",}, #These are the various styling options used
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph( #It represents a Graph
                        id="price-chart", config={"displayModeBar": False},
                    ),
                    style={"marginBottom": "24px","boxShadow": "0 4px 6px 0 rgba(0, 0, 0, 0.18)", #These are the various styling options used
},
                ),
                html.Div(
                    children=dcc.Graph( #It represents a Graph
                        id="volume-chart", config={"displayModeBar": False},
                    ),
                    style={"marginBottom": "24px","boxShadow": "0 4px 6px 0 rgba(0, 0, 0, 0.18)", #These are the various styling options used
},
                ),
            ],
            style={"marginRight": "auto","marginLeft": "auto","maxWidth": "1024px","paddingRight": "10px","paddingLeft": "10px","marginTop": "32px",}, #These are the various styling options used
        ),
    ]
)

#Call back functions get trigerred when there is a change in input and subsequently they provide with the new output
#In our Case we would be providing the share trend and duration as the inputs and would get two output plots
#Through these call back functions we are able generate a new ouput whenever we change the input parameters
@app.callback(
    [Output("price-chart", "figure"), Output("volume-chart", "figure")],
    [
        Input("trend-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(trend, start_date, end_date):
    mask = ((data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :] #Filters the data based on the duration  provided

    if trend == 'High v/s Low': #Plots the graph for the specified trend
      price_chart_figure = go.Figure() #Initialises the plot
      price_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data["High"], #Adds the data fields and mode
                    mode='lines+markers',
                    name='High'))
      
      price_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data["Low"], #Adds the data fields and mode
                    mode='lines+markers',
                    name='Low'))
      
      price_chart_figure.update_layout(title='Variation of the Highest and Lowest Price ', #Adds the graph title and the respective axis titles
                   xaxis_title='Duration',
                   yaxis_title='Price of the Share (Rs)')
      
    elif trend == 'Open v/s Close': #Plots the graph for the specified trend
      price_chart_figure = go.Figure() #Initialises the plot
      price_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data["Open"], #Adds the data fields and mode
                    mode='lines+markers',
                    name='Open',))
      
      price_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data["Close"], #Adds the data fields and mode
                    mode='lines+markers',
                    name='Close'))
      
      price_chart_figure.update_layout(title='Variation of the Opening and Closing Price ', #Adds the graph title and the respective axis titles
                   xaxis_title='Duration',
                   yaxis_title='Price of the Share(Rs)')
      
    else: #Plots the graph for the specified trend
      price_chart_figure = go.Figure() #Initialises the plot
      price_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data[trend], #Adds the data fields and mode
                    mode='lines+markers',))
      
      price_chart_figure.update_layout(title='Variation of the Share Price', #Adds the graph title and the respective axis titles
                   xaxis_title='Duration',
                   yaxis_title='Price of the Share(Rs)')

    volume_chart_figure = go.Figure() #Initialises the plot
    volume_chart_figure.add_trace(go.Scatter(x=filtered_data["Date"], y=filtered_data["Volume"], #Adds the data fields and mode
                    mode='lines+markers',))
      
    volume_chart_figure.update_layout(title='Variation of the Volume of Shares Traded', #Adds the graph title and the respective axis titles
                   xaxis_title='Duration',
                   yaxis_title='Volume of Shares Traded(millions)')
    
    return price_chart_figure, volume_chart_figure #Returns the two plots

#Through this section of the code we run our application locally using flasks built in server
if __name__ == "__main__": 
    app.run_server(mode='inline',port=8030) #Specifying mode and port

<IPython.core.display.Javascript object>